# 샘플 데이터 입력

이 노트북에서는 data/sample_products.json 파일에 있는 샘플 데이터를 Azure AI Search 인덱스에 업로드합니다.

## 0. 필요한 라이브러리 설치

In [ ]:
##############################################
# 0. 라이브러리 설치 (최초 1회 실행)
##############################################
%pip install --quiet python-dotenv azure-identity azure-search-documents openai

## 1. 환경 변수 설정

In [ ]:
import os
from dotenv import load_dotenv

##############################################
# 1. 환경 변수 로드 및 확인
##############################################
load_dotenv()

# Azure AI Search
AZURE_SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
AZURE_SEARCH_INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX_NAME")

# Azure OpenAI
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")

print(f"Search Endpoint: {AZURE_SEARCH_SERVICE_ENDPOINT}")
print(f"Search Index: {AZURE_SEARCH_INDEX_NAME}")
print(f"OpenAI Endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"Embedding Model: {AZURE_OPENAI_EMBEDDING_DEPLOYMENT}")

## 2. 샘플 데이터 로드

In [ ]:
import json

##############################################
# 2. 샘플 데이터 로드
##############################################
data_file = "data/sample_products.json"

with open(data_file, "r", encoding="utf-8") as f:
    products = json.load(f)

print(f"총 {len(products)}개의 상품 데이터를 로드했습니다.")
print(f"\n첫 번째 상품 예시:")
print(json.dumps(products[0], indent=2, ensure_ascii=False))

In [ ]:
from IPython.display import display, HTML

##############################################
# 2-1. 전체 상품 목록 시각화
##############################################
print(f"📦 전체 상품 목록 ({len(products)}개)")
print("="*80)

# HTML로 이미지와 정보를 보기 좋게 표시
html_content = '<div style="display: flex; flex-wrap: wrap; gap: 20px;">'

for idx, product in enumerate(products[:9], 1):
    html_content += f'''
    <div style="border: 1px solid #ddd; border-radius: 8px; padding: 15px; width: 250px; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
        <div style="font-weight: bold; font-size: 14px; color: #333; margin-bottom: 10px;">
            {idx}. {product["name"]}
        </div>
        <img src="{product["imageUrl"]}" style="width: 100%; height: 200px; object-fit: cover; border-radius: 4px; margin-bottom: 10px;" />
        <div style="font-size: 12px; color: #666; margin-bottom: 5px;">
            <strong>브랜드:</strong> {product["brand"]}
        </div>
        <div style="font-size: 12px; color: #666; margin-bottom: 5px;">
            <strong>가격:</strong> {product["price"]:,}원
        </div>
        <div style="font-size: 11px; color: #888; line-height: 1.4;">
            {product["description"][:50]}...
        </div>
    </div>
    '''

html_content += '</div>'

display(HTML(html_content))

## 3. 임베딩 생성 및 업로드

In [ ]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.search.documents import SearchClient
from openai import AzureOpenAI
from tqdm import tqdm

##############################################
# 3-1. Azure OpenAI 서비스 인증 설정 (임베딩용)
##############################################
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(
    credential,
    "https://cognitiveservices.azure.com/.default"
)

##############################################
# 3-2. AI Search 클라이언트 생성
##############################################
search_client = SearchClient(
    endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,
    index_name=AZURE_SEARCH_INDEX_NAME,
    credential=credential
)

##############################################
# 3-3. Azure OpenAI 클라이언트 생성
##############################################
openai_client = AzureOpenAI(
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_ad_token_provider=token_provider
)

##############################################
# 3-5. 기존 데이터 삭제 (선택사항)
##############################################
# 기존 인덱스의 모든 문서 삭제
try:
    all_docs = search_client.search(search_text="*", select=["id"])
    doc_ids = [{"id": doc["id"]} for doc in all_docs]
    if doc_ids:
        delete_result = search_client.delete_documents(documents=doc_ids)
        print(f"기존 문서 {len(doc_ids)}개 삭제 완료")
    else:
        print("삭제할 기존 문서가 없습니다.")
except Exception as e:
    print(f"기존 문서 삭제 중 오류 (무시하고 진행): {e}")


##############################################
# 3-4. 임베딩 생성 및 문서 준비
##############################################
def get_embedding(text):
    response = openai_client.embeddings.create(
        input=text,
        model=AZURE_OPENAI_EMBEDDING_DEPLOYMENT
    )
    return response.data[0].embedding

documents = []
for product in tqdm(products, desc="임베딩 생성 중"):
    embedding = get_embedding(product["description"])
    
    document = {
        "id": product["id"],
        "name": product["name"],
        "brand": product["brand"],
        "description": product["description"],
        "imageUrl": product["imageUrl"],
        "price": product["price"],
        "descriptionVector": embedding
    }
    documents.append(document)


##############################################
# 3-6. 인덱스에 업로드
##############################################
result = search_client.upload_documents(documents=documents)
print(f"\n✅ {len(documents)}개 문서 업로드 완료")
print(f"성공: {sum([1 for r in result if r.succeeded])}개")
print(f"실패: {sum([1 for r in result if not r.succeeded])}개")

## 4. 업로드 확인

In [ ]:
##############################################
# 4. 업로드 확인
##############################################
results = search_client.search(
    search_text="*", 
    top=3
)

print("업로드된 상품 샘플 (3개):")
for result in results:
    print(f"\n- ID: {result['id']}")
    print(f"  이름: {result['name']}")
    print(f"  브랜드: {result['brand']}")
    print(f"  가격: {result['price']:,.0f}원")
    print(f"  설명: {result['description']}")
    print(f"  이미지 URL: {result['imageUrl']}")
    print(f"  설명 벡터 (처음 5개 값): {result['descriptionVector'][:5]}...")